In [3]:
import create_keypair as ckp
import schnorr_lib as sl

## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes his public key $P = dG = (P_x, P_y)$

In [7]:
n_keys = int(input("Inserisci il numero di chiavi:"))

ckp.create_keypair(n_keys)["users"]

[{'privateKey': 'ed3ae4265b952f15e1139f488f24164b2e5ae14a83fbb6a9bc303d781a1f4130',
  'publicKey': 'fe6fe4545590ef5c1788e87faef481deeb04c0c2184f5209d0bf39318a7d9fef'},
 {'privateKey': 'b1407fb0a9048d980102555c3c15ee23d16f9eb4a9ee6772c08b95d559ef337d',
  'publicKey': 'c83652660670b5e313a21cad358dc8fc6a6d1011ded50525d9e184b7c13558bc'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A calculates $e = h(R || P || M)$
- A calculates $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [9]:
user = ckp.create_keypair(1)["users"]

M = input("Inserisci il messaggio:")

M = sl.sha256(M.encode())

sig = sl.schnorr_sign(M, user[0]["privateKey"])

sig.hex()

'e11a636804d81e02cdcdf06e583a14d573de3fadfe0f787558e39e45599a822fb592773a3c6de062e3b966a162e0f262987c7a6a60392f61f495747500d6b2d0'

## Verify

- $B$, the receiver, calculates $R_{v} = sG$
- $B$ calculates $e = h(R || P || M)$
- if $R_{v} - R = eP$ the signature is valid

## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(\tilde{X}̃||R||M)$
- Every user computes $s_i = r_i + c\; d_i\; a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $̃\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(\tilde{X}̃||R_1||...||R_{\nu}||M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(\tilde{X}̃||R||M)$, then every user computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

## Verify MuSig-1 and MuSig-2

- Check if $sG = R + c \tilde{X}$